# Imports

In [1]:
import aurorax
import datetime
import pprint
import os
import pandas as pd

# Upload ephemeris data

In [2]:
# read API key from environment vars
api_key = os.environ["AURORAX_API_KEY"]

In [3]:
# set values
program = "notebook-program"
platform = "notebook-platform"
instrument_type = "notebook-instrument-type"
metadata = {
    "test_meta1": "testing1",
    "test_meta2": "testing2",
}
epoch = datetime.datetime(2020, 1, 1, 0, 0)
location_geo = aurorax.Location(51.049999, -114.066666)
location_gsm = aurorax.Location(150.25, -10.75)
nbtrace = aurorax.Location(1.23, 45.6)
sbtrace = aurorax.Location(7.89, 101.23)

In [4]:
# get the ephemeris source ID
source = aurorax.sources.get_using_filters(program=[program],
                                           platform=[platform],
                                           instrument_type=[instrument_type])
identifier = source["data"][0]["identifier"]
print("Identifier: %d" % (identifier))

Identifier: 356


In [5]:
# create Ephemeris object
e = aurorax.ephemeris.Ephemeris(identifier,
                                program,
                                platform,
                                instrument_type,
                                epoch,
                                location_geo,
                                location_gsm,
                                nbtrace,
                                sbtrace,
                                metadata)
pprint.pprint(e)

{'epoch': datetime.datetime(2020, 1, 1, 0, 0),
 'identifier': 356,
 'instrument_type': 'notebook-instrument-type',
 'location_geo': Location(lat=51.049999, lon=-114.066666),
 'location_gsm': Location(lat=150.250000, lon=-10.750000),
 'metadata': {'test_meta1': 'testing1', 'test_meta2': 'testing2'},
 'nbtrace': Location(lat=1.230000, lon=45.600000),
 'platform': 'notebook-platform',
 'program': 'notebook-program',
 'sbtrace': Location(lat=7.890000, lon=101.230000)}


In [6]:
# set records array
#
# we do this because you can upload multiple ephemeris records, so the 
# standard object being passed to the upload function is a list
records = []
records.append(e)

In [7]:
# upload record
res = aurorax.ephemeris.upload(api_key, identifier, records=records)

In [8]:
# check the response to see if the upload succeeded
if (res["status_code"] == 202):
    print("Successfully uploaded")
else:
    print("Failed to upload")
    pprint.pprint(res)

Successfully uploaded


# Check that the data was uploaded by retrieving it

In [9]:
# set values
start_dt = epoch
end_dt = epoch
programs = [program]
platforms = [platform]
instrument_types = [instrument_type]

# perform search
s = aurorax.ephemeris.search(start_dt,
                             end_dt,
                             programs=programs,
                             platforms=platforms,
                             instrument_types=instrument_types,
                             show_progress=True)

[2020-11-22 19:55:32.417956] Search object created
[2020-11-22 19:55:34.776592] Request submitted
[2020-11-22 19:55:34.776749] Request ID: 136bc332-ae43-46ee-abdb-ba4f6033b773
[2020-11-22 19:55:34.776819] Request details available at: http://api.staging.aurorax.space/api/v1/ephemeris/requests/136bc332-ae43-46ee-abdb-ba4f6033b773
[2020-11-22 19:55:34.827212] Checking for data ...
[2020-11-22 19:55:37.243106] Request has data available, retrieving it ...
[2020-11-22 19:55:39.632446] Retrieved 2.1 kB of data containing 1 records, completed search request


In [10]:
# show data as a pandas dataframe
df = pd.DataFrame(s["data"])
df

,data_source,epoch,location_geo,location_gsm,nbtrace,sbtrace,metadata
0,"{'identifier': 356, 'program': 'notebook-progr...",2020-01-01,"{'lat': 51.049999, 'lon': -114.066666}","{'lat': 29.75, 'lon': -10.75}","{'lat': 1.23, 'lon': 45.6}","{'lat': 7.89, 'lon': 101.23}","{'test_meta1': 'testing1', 'test_meta2': 'test..."
